# Protocol for analysis of labeled proteomics data
We recommend the following structure for describing the protocol and its example workflow. Below you can find an overview of required features of the protocol and its description and rules for best practices.

This is the link to the github repository of the protocol 
TODO add version!
https://github.com/ProtProtocols/biocontainer-jupyter

Link to docker image:



## Abstract
Provide a short description of the software protocol including broader context, functionality, use case and purpose. 


## Maintainer
Provide details about the protocol maintainer (e.g. email address and/or github username)

## Software
Specify links for documentation and tutorials of used software, source code, publications and use cases. Detail versions of each used software. Alternatively, provide links to the software descriptions in https://bio.tools where this information is available.

## Diagram
Provide a simple diagram of functionality of the workflow/software. We recommend using controlled vocabularies for input/output data types and file formats as well as provided operation of the tool(s). You can use http://edamontology.org terms for the description.

__TODO: example__

## System requirements
Fill in the following items:
Required hard disk space for docker image, input and output files: 

Required memory: 

Recommmended number of threads: 

## Example 
Presentation of well-documented instructions and commands to run the example use case. Depending on the use case and the software, provide link(s) to open the web service incorportated in the Docker image (e.g. 0.0.0.0:8080), bash commands to run programs from the command line and additional code for e.g. checking and visualizing the (intermediate) results. 

Instead of providing the instructions in this notebook, one can also provide a link to a notebook containing the example use case.

## More general use case (optional)
Provide link to notebook with a generalized use case that easily can be adapted to e.g. process different input data and concurrent parametrization.




# Example


## TODO list

- Remove quotes from titles in mgf-files
- Add possibility of samples that are arranged in different folders
- Python script: Adapt all MGF titles prior to search
- Python script: Fix search with new PeptideShaker version
- Python script: Add MGF Peak filter function (100 - 150)
- Evaluate TMTc: https://pubs.acs.org/doi/10.1021/acs.analchem.7b04713
- Protein inference function
- Protein summarization function

Specify parameters for database search and evaluation of identified peptide-spectrum matches:

In [ ]:
%load_ext rpy2.ipython

In [ ]:
import ipywidgets as widgets
from ipywidgets import VBox, Label
import sys, os
import subprocess

# create an empty class as a storage for all UI widgets
class SearchUI:
    def __init__(self):
        self.work_dir_select = widgets.Dropdown(options={'/data/': '/data/', 'Example files': '/home/biodocker/'}, 
                                                value='/home/biodocker/')
        self.work_dir_select.observe(observe_work_dir_select)
        
        self.precursor_tolerance = widgets.IntSlider(min=-10,max=30,step=1,value=20)
        self.fragment_tolerance = widgets.BoundedFloatText(min=0,max=200,value=0.05)
        self.fasta_db = widgets.Dropdown(options={"sp_human.fasta": "IN/sp_human.fasta"})
        
        self.generate_decoy = widgets.Checkbox(value=True, description="Generate decoy sequences")
        
        # TODO  needs table to describe labeling formats
        self.labelling = widgets.Dropdown(options=
                          {'TMT6': 'TMT 6-plex of K,TMT 6-plex of peptide N-term',
                           'TMT10': 'TMT 10-plex of K,TMT 10-plex of peptide N-term','iTRAQ4': 'iTRAQ 4-plex of K,iTRAQ 4-plex of Y,iTRAQ 4-plex of peptide N-term',
                           'iTRAQ8 (fixed)': 'iTRAQ 8-plex of K, iTRAQ 8-plex of peptide N-term',
                           'iTRAQ8 (variable)': 'iTRAQ 8-plex of Y'},
                      value='TMT 10-plex of K,TMT 10-plex of peptide N-term')
        
        self.missed_cleavages = widgets.IntSlider(min=0,max=10,step=1,value=1)
        self.fixed_ptms = widgets.Dropdown(options=["Carbamidomethylation of C","None"])

        # PTMs
        self.var_ptms = widgets.SelectMultiple(
            options=["Oxidation of M",
                     "Phosphorylation of STY",
                     "Acetylation of peptide N-term",
                     "Acetylation of protein N-term"],
            value=['Oxidation of M'])
        
        self.spectra_dir = widgets.Dropdown(options={"IN": "IN"})

        # ww = widgets.Checkbox(description="Decoy")

        self.search_button = widgets.Button(
            description='Run Search',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Launch the search',
            icon='check'
        )

        self.search_button.on_click(run_search)
        
        
    def updateFastaFiles(self, workdir):
        # get all FASTA files
        fasta_files = [file for file in os.listdir(workdir) if file[-6:] == ".fasta"]
        
        # also search all subdirectories for FASTA files
        for d in os.listdir(workdir):
            d_path = os.path.join(workdir, d)
            if os.path.isdir(d_path) and d[0] != ".":
                fasta_files += [os.path.join(d, file) for file in os.listdir(d_path) if file[-6:] == ".fasta"]
        
        # create the dict to add as values to the control
        file_list = dict()
        sel_value = None
        
        for f in fasta_files:
            file_list[f] = os.path.join(os.path.abspath(workdir), f)
            if sel_value is None:
                sel_value = os.path.join(os.path.abspath(workdir), f)
        
        self.fasta_db.options = file_list
        self.fasta_db.value = sel_value
        
        # update the list of possible peaklist directories
        directories = [d for d in os.listdir(workdir) if os.path.isdir(os.path.join(workdir, d)) and d[0] != "."]
        
        dir_list = dict()
        
        for d in directories:
            dir_list[d] = os.path.join(os.path.abspath(workdir), d)
            
        self.spectra_dir.options = dir_list
        
        if "IN" in dir_list:
            self.spectra_dir.value = dir_list["IN"]
        
        
    def display(self):
        self.updateFastaFiles(self.work_dir_select.value)
        
        settings_box = VBox([Label('Working directory'), self.work_dir_select,
                             Label('Precursor tolerance (ppm):'), self.precursor_tolerance, 
                             Label('Fragment ion tolerance (da):'), self.fragment_tolerance,
                             Label('Fasta file (database, must NOT contain decoy sequences):'), self.fasta_db,
                             self.generate_decoy,
                             Label('Quantification method:'), self.labelling,
                             Label('Number of miscleavages;'), self.missed_cleavages,
                             Label('Further fixed modifications'), self.fixed_ptms,
                             Label('Further variable modifications (Hold Ctrl to select multiple)'), self.var_ptms,
                             Label('Folder for spectra files (files need to be mgf)'), self.spectra_dir,
                             self.search_button])

        display(settings_box)
        

class ExpDesignUI:
    def __init__(self, labelling_technique):
        """
        Generates all use interface objects as member variables.
        
        :param labelling_technique: The labelling method used.
        """
        # always expect two groups
        self.group1_name = widgets.Text(placeholder = "Treatment", description = "Group 1:")
        self.group2_name = widgets.Text(placeholder = "Control", description = "Group 2:")
        
        self.channels = {
            'TMT6': ["126", "127", "128", "129", "130", "131"],
            'TMT10': ["126", "127N", "127C", "128N", "128C", "129N", "129C", "130N", "130C", "131"],
            'iTRAQ8': ["113", "114", "115", "116", "117", "118", "119", "121"]
        }
        
        if labelling_technique not in self.channels:
            raise Exception("Unknown labelling technique: '" + labelling_technique + "'")
            
        self.labelling_technique = labelling_technique
            
        # generate the textfields for the channels
        self.channel_names = list()
        
        for channel in self.channels[self.labelling_technique]:
            self.channel_names.append(widgets.Text(description = channel, placeholder = "Sample " + channel))
            
        # add select boxes to select the experimental group
        self.group_selects = list()
        
        for channel in self.channels[self.labelling_technique]:
            self.group_selects.append(widgets.Dropdown(options = ["Group 1", "Group 2"], value = "Group 1"))
            
        self.save_button = widgets.Button(
            description='Save design',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Save the experimental design',
            icon='check'
        )

        self.save_button.on_click(self.save_design)

    def display(self):
        widget_list = [widgets.Label("Treatment group names:"), self.group1_name, self.group2_name,
                       widgets.Label("Sample names (per channel):")]
        
        for i in range(0, len(self.channel_names)):
            widget_list.append(widgets.HBox([self.channel_names[i], self.group_selects[i]]))
            
        widget_list.append(self.save_button)
        
        widget_box = VBox(widget_list)
        
        display(widget_box)
        
    def save_design(self):
        pass
            
        
def run_search(button):
    global searchUI, result_file
    
    # make sure all required fields were selected
    if searchUI.work_dir_select.value is None:
        print("Error: No working directory selected")
        return
    
    if searchUI.fasta_db.value is None:
        print("Error: No FASTA file selected")
        return
    
    # create the directory paths to work in
    peaklist_dir = os.path.abspath(searchUI.spectra_dir.value)
    
    if not os.path.isdir(peaklist_dir):
        raise Exception("Invalid peak list directory selected: " + peaklist_dir + " does not exist.")
    
    peptide_shaker_jar = "/home/biodocker/bin/PeptideShaker-1.16.17/PeptideShaker-1.16.17.jar"
    searchgui_jar = "/home/biodocker/bin/SearchGUI-3.2.20/SearchGUI-3.2.20.jar"
    work_dir = os.path.abspath(os.path.join(searchUI.work_dir_select.value, "OUT"))
    
    # the searches should be performed in the "OUT" directory
    if not os.path.isdir(work_dir):
        os.mkdir(work_dir)
        
    # -------------------------------------
    # Generate the decoy database
    
    if searchUI.generate_decoy.value == True:
        print("Creating decoy database...")

        # create the decoy database
        subprocess.run(["java", "-cp", searchgui_jar, 
                       "eu.isas.searchgui.cmd.FastaCLI", "-in", searchUI.fasta_db.value, "-decoy"], check=True,
                       cwd=work_dir,
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # get the filename of the decoy database
        database_file = os.path.abspath(searchUI.fasta_db.value)[:-6] + "_concatenated_target_decoy.fasta"
    else:
        # simply use the selected database file
        database_file = os.path.abspath(searchUI.fasta_db.value)
    
    if not os.path.isfile(database_file):
        raise Exception("Failed to find generated decoy database")
        
    # ---------------------------------------------
    # Create the search parameter file
        
    # build the arguments to create the parameter file
    param_file = os.path.join(work_dir, "search.par")
    
    # remove any old parameters
    if os.path.isfile(param_file):
        os.remove(param_file)
    
    search_args = ["java", "-cp", searchgui_jar,
                   "eu.isas.searchgui.cmd.IdentificationParametersCLI",
                   "-out", param_file]
    
    # precursor tolerance
    search_args.append("-prec_tol")
    search_args.append(str(searchUI.precursor_tolerance.value))
    # fragment tolerance
    search_args.append("-frag_tol")
    search_args.append(str(searchUI.fragment_tolerance.value))
    # fixed mods
    # TODO: labelling cannot always be set as fixed mod???
    fixed_mod_string = str(searchUI.labelling.value) + "," + str(searchUI.fixed_ptms.value)
    search_args.append("-fixed_mods")
    search_args.append(fixed_mod_string)
    # database
    search_args.append("-db")
    search_args.append(database_file)
    # missed cleavages
    search_args.append("-mc")
    search_args.append(str(searchUI.missed_cleavages.value))
    
    # var mods
    if len(searchUI.var_ptms.value) > 0:
        search_args.append("-variable_mods")
        var_mod_list = list()
        
        for var_mod in searchUI.var_ptms.value:
            if var_mod == "Phosphorylation of STY":
                var_mod_list += ["Phosphorylation of S", "Phosphorylation of T", "Phosphorylation of Y"]
            else:
                var_mod_list.append(var_mod)
                
        search_args.append(",".join(var_mod_list))
        
    # create the search parameter file
    print("Creating search parameter file...")
    # print(" ".join(search_args))
    subprocess.run(search_args, check=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if not os.path.isfile(param_file):
        raise Exception("Failed to create search parameters")
        
    # ------------------------------------------------
    # Run the search
    print("Running search...")
    # TODO: create list of spectrum files - or the folder
    spectrum_files = peaklist_dir
    print("  Searching files in " + spectrum_files)
    search_process = subprocess.run(["java", "-cp", searchgui_jar,
                    "eu.isas.searchgui.cmd.SearchCLI", "-spectrum_files", spectrum_files,
                    "-output_folder", work_dir, "-id_params", param_file,
                    "-xtandem", "0", "-msgf", "1", "-comet", "0", "-ms_amanda", "0", 
                    "-myrimatch", "0", "-andromeda", "0", "-omssa", "0", "-tide", "0"],
                    check=False, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    
    if search_process.returncode != 0:
        print(search_process.stdout)
        raise Exception("Search process failed.")
    
    print("Search completed.")
    
    # -------------------------------------------------
    # Run PeptideShaker
    print("Processing result using PeptideShaker...")
    peptide_shaker_result_file = os.path.join(work_dir, "experiment.cpsx")

    peptide_shaker_process = subprocess.run(["java", "-cp", peptide_shaker_jar,
                    "eu.isas.peptideshaker.cmd.PeptideShakerCLI",
                    "-experiment", "experiment1",
                    "-sample", "test",
                    "-replicate", "1",
                    "-identification_files", work_dir,
                    "-out", peptide_shaker_result_file,
                    "-id_params", param_file,
                    "-spectrum_files", spectrum_files],
                    check=False, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    
    if peptide_shaker_process.returncode != 0:
        print(peptide_shaker_process.stdout)
        raise Exception("Failed to run PeptideShaker")

    if not os.path.isfile(peptide_shaker_result_file):
        raise Exception("Failed to process result file.")
      
    # ---------------------------------------------------
    # create TSV output files
    print("Converting result to TSV format...")
    conversion_process = subprocess.run(["java", "-cp", peptide_shaker_jar,
                  "eu.isas.peptideshaker.cmd.ReportCLI",
                  "-in", peptide_shaker_result_file,
                  "-out_reports", work_dir,
                  "-reports", "8"],
                  check=False, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    
    if conversion_process.returncode != 0:
        print(conversion_process.stdout)
        raise Exception("Conversion process failed")
    
    result_file=os.path.join(work_dir, "experiment1_test_1_Extended_PSM_Report.txt")
    
    if not os.path.isfile(result_file):
        raise Exception("Error: Conversion failed")
        
    print("Done.")
    
    # -----------------------------------------------------
    # Create the experimental design
    labelling_method = list(searchUI.labelling.options.keys())[searchUI.labelling.index]
    
    if labelling_method[0:6] == "iTRAQ8":
        labelling_method = "iTRAQ8"
        
    expDesignUI = ExpDesignUI(labelling_method)
    expDesignUI.display()
    
    
def observe_work_dir_select(change):
    global work_dir_select
    if change['type'] == "change" and change['name'] == "value":
        # update the required UI controls
        searchUI.updateFastaFiles(change["new"])

# -------------------
# Code to create the UI
# --------------------
result_file=None
work_dir=None
peaklist_dir=os.path.abspath("IN")

searchUI = SearchUI()
searchUI.display()

# create parameter list as input for R script
Rinput = [searchUI.labelling.value, searchUI.spectra_dir.value]

#TODO set names of samples and replicates (peptideshaker)

In [ ]:
%%R -i Rinput

# reading parameters
labeling <- Rinput[[1]]
inputDir <- Rinput[[2]]

# library causes the execution to fail if the library is missing
suppressWarnings(suppressMessages(library(lattice)))
suppressWarnings(suppressMessages(library(stringr)))
suppressWarnings(suppressMessages(library(mzID)))
#suppressWarnings(suppressMessages(library(matrixStats)))
#suppressWarnings(suppressMessages(library(venneuler)))
suppressWarnings(suppressMessages(library(MSnbase)))

# IMPORTANT
# remove quotes from mgf-files before conversion:
#for file in *mgf; do sed 's/\"//g' $file > ${file%.*}_noquote.mgf; done
#for file in *_noquote.mgf; do /usr/local/tpp/bin/msconvert "$file" "${file%.*}.mzML"; done 
# Add option to provide mzML directly

print(inputDir)
setwd(inputDir)

## Folder names for different runs (e.g. different replicates, ...)
samples <- s <- c(".")
ProtDat <- list()
for (s in samples) {
  
  ## Organize technical runs in the same folder
  # filenames
  ident_files <- list.files(paste("/home/biodocker/OUT/",s,sep=""),pattern="Extended_PSM_Report.txt$",full.names=T)
  #mzML_files <- list.files(s,pattern=".mzML$",full.names = T)
  #mgf_files <- list.files(s,pattern="_noquote.mgf$",full.names = T)
  mgf_files <- list.files(s,pattern="mgf$",full.names = T)
  
  # TODO load parameters "%%R -i parname"
  # process the input files
  max.fdr <- 0.01
  
  # translate into labeling method descriptors
  quant.methods <- cbind(c("TMT10-plex of peptide N-term","TMT 6-plex of K,TMT 6-plex of peptide N-term",
                           "iTRAQ 4-plex of K,iTRAQ 4-plex of Y,iTRAQ 4-plex of peptide N-term",
                           "iTRAQ 8-plex of K, iTRAQ 8-plex of peptide N-term","iTRAQ 8-plex of Y"),
                         c(TMT10,TMT6,iTRAQ4,iTRAQ8,iTRAQ8))
  quant.method <- quant.methods[quant.methods[,1] == labeling,2]
  ## TODO read from experimental design
  class.labels <- c("A", "B", "C", "D", "E", "F", "G", "H", "I", "J")
  args <- commandArgs(trailingOnly = TRUE)
  
  if (!file.exists(ident_files)) {
    stop("Error: Cannot find identification file ", ident_filess)
  }
  for (mgf_file in mgf_files) {
    if (!file.exists(mgf_file)) {
      stop("Error: Cannot find mgf file ", mgf_file)
    }
  }
  
  # Convert SearchGUI output to isobar format
  psms <- read.csv(ident_files, sep = "\t",stringsAsFactors = F)
  
  if (! "Decoy" %in% names(psms)) {
    stop("Error: No decoy information available in output file")
  }
  
  print(paste("Loaded",nrow(psms), "PSMs"))
  
  # ---- Confidence filter ----
  psms <- psms[order(psms[, "Confidence...."], decreasing = T), ]
  decoy.psms <- which(psms[, "Decoy"] == "1")
  
  decoy.count <- 0
  
  for (decoy.index in decoy.psms) {
    decoy.count <- decoy.count + 1
    target.count <- decoy.index - decoy.count
    
    cur.fdr <- (decoy.count * 2) / (decoy.count + target.count)
    
    if (cur.fdr > max.fdr) {
      # filter
      psms <- psms[1:decoy.index - 1,]
      break
    }
  }
  
print(paste0("Filtered ", nrow(psms), " PSMs @ ", max.fdr, " FDR"))
    
    print(head(psms))
    
  #### prepare for MSnbase
  psms$rank <- 1
  psms$desc <- psms$Protein.s.
  psms$spectrumID <- psms$Spectrum.Title #str_extract(psms$Spectrum.Title, "scan=[0-9]*")
  psms$spectrumFile <- psms$Spectrum.File
  psms$idFile <- ident_files
  # remove unnecessary PTMs from modified sequence
  # TODO: define whether to take oxidation, ...
  psms$Modified.Sequence <- gsub("<cmm>","",psms$Modified.Sequence)
  psms$Modified.Sequence <- sub("[a-z,A-Z]*-","",psms$Modified.Sequence)
  psms$Modified.Sequence <- sub("-[a-z,A-Z]*","",psms$Modified.Sequence)
  psms$Modified.Sequence <- gsub("<iTRAQ>","",psms$Modified.Sequence)
  psms$Modified.Sequence <- gsub("<TMT>","",psms$Modified.Sequence)
  psms$sequence <- psms$Modified.Sequence
    #Optional:
  # psms$Modified.Sequence <- gsub("<ox>","",psms$Modified.Sequence)
  
  
  
  # reading MS and identification data
  allSpectra <- list()
  for (mgf_file in mgf_files) {
    # myExp1 <- readMSData(mzML_files, mode="onDisk", verbose=T)
    
    # trim mgf file by removing all masses below 130
    system(paste("awk '$1<130 || !($1 ~ /^[0-9,\\.]+/)'", mgf_file, "> t.mgf"))
    
    
    myExp1 <- readMgfData("t.mgf", verbose=T)
    # myExp1 <- MSnbase::readMgfData("t.mgf", verbose=T)
    system("rm t.mgf")
    for (i in 1:ncol(fData(myExp1))) {
      if (is.factor(fData(myExp1)[,i]))
        fData(myExp1)[,i] <- as.character(fData(myExp1)[,i])
    }
    fData(myExp1)$spectrumFile <- mgf_file
    allSpectra[[mgf_file]] <- myExp1 #updateFeatureNames(myExp1,label = paste("Sample",i))
  }

  
  
  # myExp1 <- addIdentificationData(myExp1,psms,decoy="Decoy",rank="rank",acc="Protein.s.",
  #                                 icol="spectrumID",fcol="spectrumId",desc="desc",pepseq="Modified.Sequence",verbose=T)
  cl <- makeCluster(4)
  myExp <- qnt <- list()
  for (mgf_file in mgf_files) {
    myExp[[mgf_file]] <- addIdentificationData(allSpectra[[mgf_file]],psms,decoy="Decoy",rank="rank",acc="Protein.s.",
                                  icol="Spectrum.Title",fcol="TITLE",desc="desc",pepseq="Modified.Sequence",verbose=T)
    print(idSummary(myExp[[mgf_file]]))
    myExp[[mgf_file]] <- removeNoId(myExp[[mgf_file]])
    qnt[[mgf_file]] <- quantify(myExp[[mgf_file]], method="max", reporters = quant.method, strict =F, verbose=T )
  } 

  ## Convert SearchGUI output to file format that can be processed by isobar
  stopCluster(cl)
  
  imp<-makeImpuritiesMatrix(length(quant.method[[1]]),edit=F)
  for (i in 1:length(qnt)) {
    qnt[[i]] <- purityCorrect(qnt[[i]],imp)
    exprs(qnt[[i]]) <- log2(exprs(qnt[[i]]))
    qnt[[i]] <- normalise(qnt[[i]],"center.median")
    qnt[[i]] <- updateFeatureNames(qnt[[i]],label = paste("Sample",i))
  }
  
  names(qnt) <- NULL
  allqnt <- do.call("combine",args=qnt)
  
  allqnt <- filterNA(allqnt, pNA=0.5 )
  
  ## Here we could start using different methods
  allProts <- combineFeatures(allqnt, groupBy=fData(allqnt)$Protein.s., fun="medpolish",na.rm=T)
  
  boxplot(exprs(allProts))
  
  pairs(exprs(allProts))
  #save
  write.exprs(allProts,file=paste(sample,"/ProtData.csv",sep=""),sep=",")
  
  ProtDat[[s]] <- allProts <- exprs(allProts)
  pca <- princomp(allProts[complete.cases((allProts)),])
  plot(pca)
  plot(pca$loadings)
  text(pca$loadings,colnames(ProtDat))
  
}